In [97]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import CountVectorizer

In [76]:
J1939_faults = pd.read_csv('data/J1939Faults.csv')
service_fault_codes = pd.read_csv('data/Service_Fault_Codes_1_0_0_167.csv')
VDOD = pd.read_csv('data/VehicleDiagnosticOnboardData.csv')
updated_faults = pd.read_csv('updatedFaults.csv')

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_23564\915579820.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  J1939_faults = pd.read_csv('data/J1939Faults.csv')


In [78]:
J1939_faults.shape

(1187335, 20)

In [79]:
to_drop = [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]

# drop rows with the specified latitude and longitude values
for lat, long in to_drop:
    J1939_faults.drop(J1939_faults[(J1939_faults['Latitude'] == lat) & (J1939_faults['Longitude'] == long)].index, inplace=True)

# print the resulting dataframe
J1939_faults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187260 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   RecordID               1187260 non-null  int64  
 1   ESS_Id                 1187260 non-null  int64  
 2   EventTimeStamp         1187260 non-null  object 
 3   eventDescription       1126423 non-null  object 
 4   actionDescription      0 non-null        float64
 5   ecuSoftwareVersion     891224 non-null   object 
 6   ecuSerialNumber        844258 non-null   object 
 7   ecuModel               1122507 non-null  object 
 8   ecuMake                1122507 non-null  object 
 9   ecuSource              1187260 non-null  int64  
 10  spn                    1187260 non-null  int64  
 11  fmi                    1187260 non-null  int64  
 12  active                 1187260 non-null  bool   
 13  activeTransitionCount  1187260 non-null  int64  
 14  faultValue        

In [80]:
J1939_faults['EquipmentID'] = J1939_faults['EquipmentID'].astype(str)

allId = J1939_faults['EquipmentID'].unique().tolist()

truckId = []
for x in range(0, len(allId)):
    if len(allId[x]) < 5:
        truckId.append(allId[x])
    else:
        pass
    
truckId

['1439',
 '1369',
 '1674',
 '1417',
 '1597',
 '1429',
 '1582',
 '309',
 '1601',
 '302',
 '310',
 '1733',
 '1590',
 '1605',
 '1644',
 '1585',
 '1634',
 '1442',
 '1404',
 '1620',
 '1721',
 '1610',
 '1576',
 '1395',
 '1766',
 '1437',
 '305',
 '1586',
 '1600',
 '1558',
 '1630',
 '1646',
 '1595',
 '1432',
 '1377',
 '1641',
 '1469',
 '1371',
 '1365',
 '1677',
 '1609',
 '1612',
 '1768',
 '1650',
 '1657',
 '1514',
 '1703',
 '1451',
 '1625',
 '1565',
 '1520',
 '1623',
 '1363',
 '1569',
 '1390',
 '1607',
 '1776',
 '1370',
 '1474',
 '1673',
 '1749',
 '1658',
 '1449',
 '1566',
 '1499',
 '1427',
 '1688',
 '1467',
 '1669',
 '1606',
 '1462',
 '1553',
 '1683',
 '1391',
 '1458',
 '1727',
 '1632',
 '1778',
 '1559',
 '1431',
 '1372',
 '1769',
 '1492',
 '1434',
 '1611',
 '1402',
 '1460',
 '1481',
 '1555',
 '1375',
 '1651',
 '1419',
 '1580',
 '1374',
 '301',
 '1401',
 '1614',
 '1737',
 '1358',
 '1740',
 '1562',
 '1339',
 '1715',
 '1405',
 '1654',
 '1594',
 '1729',
 '1571',
 '1337',
 '1560',
 '1455',
 '1649

In [81]:
J1939_faults.loc[J1939_faults['EquipmentID'].isin(truckId)].reset_index(drop=True)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185063,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1185064,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1185065,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1185066,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [82]:
J1939_faults['active'] = J1939_faults['active'].astype('int')

In [83]:
J1939_faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,1,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,1,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,0,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,1,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,0,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [84]:
J1939_faults['EquipmentID'] = J1939_faults['EquipmentID'].astype('str')

In [85]:
trucks5246 = J1939_faults.loc[J1939_faults['spn'] == 5246]['EquipmentID'].unique().tolist()

trucks1569 = J1939_faults.loc[(J1939_faults['spn'] == 1569) & (J1939_faults['fmi'] == 31)]['EquipmentID'].unique().tolist()

In [86]:
vdiag_piv = VDOD.pivot(index = 'FaultId', columns = 'Name', values='Value').reset_index() 

faults_diagnostics = pd.merge(vdiag_piv, J1939_faults, how = 'inner', left_on='FaultId', right_on='RecordID' )

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,111,17,1,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,629,12,1,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1807,2,0,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1807,2,1,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4364,17,0,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187255,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1761,17,0,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187256,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,...,1569,31,1,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187257,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,...,3216,10,1,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187258,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,...,111,18,1,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [91]:
updated_faults_List = updated_faults['RecordID'].to_list()

In [95]:
updated_faults = faults_diagnostics.loc[faults_diagnostics['RecordID'].isin(updated_faults_List)]

In [96]:
updated_faults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1054113 entries, 0 to 1187259
Data columns (total 45 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   FaultId                    1054113 non-null  int64  
 1   AcceleratorPedal           479193 non-null   object 
 2   BarometricPressure         529391 non-null   object 
 3   CruiseControlActive        519277 non-null   object 
 4   CruiseControlSetSpeed      520327 non-null   object 
 5   DistanceLtd                529249 non-null   object 
 6   EngineCoolantTemperature   529379 non-null   object 
 7   EngineLoad                 528917 non-null   object 
 8   EngineOilPressure          529499 non-null   object 
 9   EngineOilTemperature       527871 non-null   object 
 10  EngineRpm                  529813 non-null   object 
 11  EngineTimeLtd              525550 non-null   object 
 12  FuelLevel                  454011 non-null   object 
 13  FuelLtd     

In [87]:
updated_faults[spn, fmi, active, ]

TypeError: '<' not supported between instances of 'str' and 'int'